In [3]:
import unicodedata
from collections import defaultdict

In [4]:
with open("../data/znaki_wikipedii.txt", "r") as f:
    chars = f.read()

In [5]:
def group_by_type(chars):
    type_dict = defaultdict(list)
    for ch in chars:
        type_dict[unicodedata.category(ch)[0]].append(ch)
    for k, v in type_dict.items():
        type_dict[k] = list(set(v))
    return type_dict

In [6]:
types = group_by_type(chars)
for k,v in types.items():
    print(k)
    print(len(v))

S
675
Z
16
L
10965
P
197
N
335
C
75
M
544


In [7]:
def group_by_code(type_dict, type, gap1, gap2):

    chars = type_dict[type]
    chars.sort(key=lambda n: ord(n))

    idx = 0
    batch = 0
    grouped = defaultdict(list)

    while idx < len(chars) - 1:
        curr = chars[idx]
        next = chars[idx + 1]
        grouped[batch].append(curr)

        if (ord(next) - ord(curr)) > gap1 and (ord(next)/ord(curr)) > gap2:
            batch += 1
        idx += 1

    return grouped

In [8]:
def name_groups(grouped, type):
    named = defaultdict(list)
    for k in grouped:
        group = grouped[k]
        min = ord(group[0])
        max = ord(group[-1])
        named[type + ":" + str(min) + "-" + str(max)] = group
    return named

In [9]:
def prepare_file(chars, path):
    f = open(path, 'w')
    types = group_by_type(chars)
    big_categories = ["L"]
    for codetype in big_categories:
        grouped_letters = group_by_code(types, codetype, 100, 1.05)
        named_letters = name_groups(grouped_letters, codetype)
        for k, g in named_letters.items():
            f.write(k)
            f.write("\n")
            f.write(" ".join(list(g)))
            f.write("\n")
    for k, g in types.items():
        if k not in big_categories:
            f.write(k)
            f.write("\n")
            f.write(" ".join(list(g)))
            f.write("\n")

In [10]:
prepare_file(chars, "../other/output2.txt")